In [1]:
!pip install git+https://github.com/SKT-AI/KoBART #egg=kobart

  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-req-build-yc_l605j
  Running command git clone -q https://github.com/SKT-AI/KoBART /tmp/pip-req-build-yc_l605j
     |████████████████████████████████| 132 kB 5.3 MB/s 
     |████████████████████████████████| 814 kB 35.8 MB/s 
     |████████████████████████████████| 776.8 MB 15 kB/s 
     |████████████████████████████████| 1.9 MB 43.3 MB/s 
     |████████████████████████████████| 829 kB 45.6 MB/s 
     |████████████████████████████████| 134 kB 42.7 MB/s 
     |████████████████████████████████| 596 kB 58.7 MB/s 
     |████████████████████████████████| 3.3 MB 49.5 MB/s 
     |████████████████████████████████| 895 kB 46.8 MB/s 
     |████████████████████████████████| 1.1 MB 46.7 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 55.1 MB/s 
     |████████████████████████████████| 144 kB 66.2 MB/s 
     |████████████████████████████████| 79 kB 9.0 MB/s 
     |█████████████████████

In [24]:
import torch
import os 
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm

import re

from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
import matplotlib.pyplot as plt

import seaborn as sns

In [3]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### 원본 데이터 불러오기

In [43]:
csv_test = pd.read_csv("/content/drive/MyDrive/NLP/sports_news_data - sports_news_data.csv")

In [44]:
csv_test.head(20)

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15
5,"긴급 수혈된 바르사 NO.9, 1년 반 만에 떠난다… ‘EPL행 유력’",[스포탈코리아] FC 바르셀로나는 새 시즌을 앞두고 선수단 정리가 한창이다. 잉여 ...,2021-07-15
6,"[김남구의 유럽통신] 황의조, 손흥민 소속사와 손잡다… CAA Base와 계약",[스포탈코리아=파리(프랑스)] 황의조(지롱댕 드 보르도)가 한국 선수로는 3번째로 ...,2021-07-15
7,"""메시 종신은 축복!""…스폰서 5년 더 보장, 바르셀로나 함박웃음",[스포탈코리아] 리오넬 메시(34)가 FC바르셀로나에 남는다. 연봉을 절반 삭감하지...,2021-07-15
8,"[오피셜] 눈물 흘렸던 '37세 전설' 로번, 두 번째 현역 은퇴 발표",[스포탈코리아] 네덜란드 축구스타 아르연 로번(37)이 현역 은퇴를 밝혔다. \n\...,2021-07-15
9,"100세' 메시팬 할아버지, 748골 수기 작성…메시도 감사 인사",[스포탈코리아] 리오넬 메시(34)는 프로 데뷔하고 748골을 터뜨렸다. 전산화 하...,2021-07-15


### 전처리
- 중복 및 결측치 제거
- 크롤링 상에서 생긴 쓸모 없는 문구 처리


In [45]:
def preprocess(string):
    str_list = string.split('\n\n') # \n\n 기준으로 분리
    str_list[0] = str_list[0].replace('[스포탈코리아] ', '') # [스포탈코리아] 삭제
    
    result = []
    for strg in str_list:
        strg = re.sub(r'\([^)]*\)|\[[^)]*\]|\<[^)]*\>', ' ', strg) # 괄호와 그 안에 문자 제거

        if '기자' in strg: # 기자 이름이 들어간 경우 기자이름 삭제
            strg = strg.split('기자')[-1]
        if '다.' in strg: # 문장 단위로 분리
            strg = strg.split('. ')
            for s in strg:
                if s != '':
                    result.append(s)
            
    return result

In [46]:
def special(str_list):
    regex = r"[^가-힣a-zA-Z0-9 ]" # 특수 문자 제거
    for i in range(len(str_list)):
        str_list[i] = re.sub(regex, '', str_list[i])

    return str_list

In [47]:
def special_title(string):
    regex = r"[^가-힣a-zA-Z0-9 ]" # 특수 문자 제거
    string = re.sub(regex, '', string)

    return string

In [48]:
trimmed_data=csv_test.dropna().copy()
  
idx = trimmed_data['CONTENT'].drop_duplicates().index
trimmed_data = trimmed_data.loc[idx]

idx = trimmed_data['TITLE'].drop_duplicates().index
trimmed_data = trimmed_data.loc[idx]

trimmed_data['CONTENT'] = trimmed_data['CONTENT'].apply(preprocess)
trimmed_data['CONTENT'] = trimmed_data['CONTENT'].apply(special)
trimmed_data['TITLE'] = trimmed_data['TITLE'].apply(special_title)


In [49]:
len(trimmed_data)

8995

In [50]:
trimmed_data['CONTENT'].iloc[8991]

['성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다',
 '김지수는 192cm84kg의 체격에 제공권이 좋고 시야가 넓어 대인 방어와 패스에 능하다',
 ' 김지수는 성남FC U15 U18에서 활약하며 성남의 유망주로 이름을 알렸으며 꾸준히 연령별 대표팀에도 이름을 올리고 있다',
 'U15 크로아티아 국제 축구대회와 AFC U16 챔피언십 예선 참여 2021년 경기도 꿈나무 축구대회 우승 2021년 제29회 백록기 전국 고교 축구대회에서 우승 주역으로 베스트 영플레이어상을 수상한 김지수는 성남FC U18 풍생고에서 성장 가능성이 큰 선수로 꼽혀왔다 구단은 유소년 육성에 집중 투자하면서 구단 유스 프로 계약의 증가와 구단 최초 준프로 계약 등 점차 가시적인 성과를 내고 있으며 꾸준한 유소년 시스템 투자와 유망주 발굴을 통해 구단의 지속적 성장의 기틀을 다질 예정이다 김남일 감독은 풍생고 경기를 꾸준히 보면서 김지수를 눈여겨봤었다',
 '책임감 있고 안정적인 경기 운영을 하는 것이 눈에 띄었다',
 '김지수가 성남에서 첫 프로 도전을 하는 만큼 열심히 준비해서 경기장에서 본인의 잠재력을 마음껏 펼치길 바란다라고 밝혔다 김지수는 이번 1차 전지훈련에서 프로팀 형들과 훈련을 한 것도 영광이었는데 바로 계약까지 하게 된 게 아직 꿈만 같다',
 '성남 유스 출신 선수라는 책임감이 생겼다',
 '2차 전지훈련 동안 선수 형들에게 많이 배우고 열심히 준비해서 팀에 도움이 되고 싶다라고 밝혔다 그는 성남FC 협력병원인 분당베스트병원에서 메디컬테스트를 마쳤고 2차 전지훈련지인 부산 기장으로 출발하여 본격적인 2022시즌 준비에 나선다']

- 띄어쓰기 및 불용어 처리

In [51]:
!pip install konlpy

In [52]:
# 한국어 불용어 리스트 크롤링
import requests
from bs4 import BeautifulSoup

url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ranks.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


# Korean stop words: 677


In [53]:
from konlpy.tag import Okt
okt = Okt()
for i in tqdm(range(len(trimmed_data))):
  temp_data = okt.morphs(trimmed_data["TITLE"].iloc[i])
  temp_list = []

  for word in temp_data:
    if word in stop_words: continue
    temp_list.append(word)
  
  trimmed_data["TITLE"].iloc[i] = " ".join(temp_list)

  temp_list = []
  for sentence in trimmed_data["CONTENT"].iloc[i]:
    temp_data = okt.morphs(sentence)
    temp_sentecne_list = []
    
    for word in temp_data:
      if word in stop_words: continue
      temp_sentecne_list.append(word)
    
    temp_sentence = " ".join(temp_sentecne_list)
    temp_list.append(temp_sentence)
  
  trimmed_data["CONTENT"].iloc[i] = temp_list
  


100%|██████████| 8995/8995 [08:40<00:00, 17.28it/s]


In [54]:
trimmed_data['CONTENT'].iloc[-1]

['이름 값 만 화려한 맨유 워스트 11 공개 됐다',
 '영국 매체 익스 프레 스 는 7일 알렉스 퍼거슨 경 은퇴 후 맨유 워스트 11 선정 했다',
 '퍼거슨 경 은퇴 후 맨유 워스트 11 에는 이름 값 만 화려한 선수 많았다',
 '공 격진 에는 멤피스 데 파이 라다 멜 팔카오 알 렉시스 산체스 포함 됐다',
 '데 파이는 많은 기대 받고 PSV 아인트호벤 떠나 맨유 유니폼 입었다',
 '보여준 없었다',
 '최악 경기 력 선보이면서 실패 작 낙인 찍혔다',
 '인간계 최강 이라고 불렸던 팔카오 도 맨유 쓴맛 봤다',
 '공 격진 무게 감 실어 줄 이라는 엄청난 기대는 산산조각 났다',
 '산체스 도 맨유 역대 최악 입 중 한 명 꼽힌다',
 '주급 7억 이라는 명성 맞지 않는 경기 력 질타 피하 지 못 했다',
 '레알 마드리드 합류 한 앙헬 디 마리아 아약스 엄청난 퍼포먼스 업고 이적 한 도니 판 더 비크 도 맨유 워스트 11 불명예 안았다',
 '이외 프레드 마 테오 다르 미안 해리 이어 타일러 블랙 켓 마르코스 로호 빅토르 발데스 이름 올렸다']

In [55]:
trimmed_data.to_csv('/content/drive/MyDrive/NLP/trimmed_sport_data.csv', sep=',', na_rep='NaN', index = False) # do not write index

### 수정한 파일 불러오기

In [22]:
trimmed_data = pd.read_csv("/content/drive/MyDrive/NLP/trimmed_sport_data.csv")

In [23]:
trimmed_data.head()

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,['유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 ...,2021-07-15
1,디 마리아 없다 유로 X 코파 베스트 11 이탈리아 만 7 명,"['지난달 시작 된 유로 코파 아메리카 11일 끝 막 내렸다', '이탈리아 는 결승...",2021-07-15
2,슈퍼컴퓨터 예측 맨시티 우승 맨유 4 위 토트넘 은 6 위,"['새 시즌 시작 하기도 전 슈퍼컴퓨터 예상 한 순위 나왔다', '영국 매체 스포츠...",2021-07-15
3,이재성 완벽한 프로 마인츠 서 성공할 킬 디렉터 애정 듬뿍 응원,"['홀슈타인 킬 우베 스토 버 디렉터 이재성 향 해 응원 메시지 띄웠다', '이재성...",2021-07-15
4,홈킷 딴판 바르사 팬 NEW 웨이 셔츠 호평 가장 좋아하는 색,"['FC 바르셀로나 새 시즌 원정 유니폼 공개 했다', '팬 은 만족스럽다는 반응 ...",2021-07-15


In [56]:
data_row = trimmed_data.iloc[0]
print(data_row)

text = data_row['CONTENT']
print(text)

for i, sentence in enumerate(text):
  print(i)
  print(sentence)

TITLE                              스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아
CONTENT       [유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나...
PUBLISH_DT                                           2021-07-15
Name: 0, dtype: object
['유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나온 판정 논란 정심 이라고 공언 했다', '지난 8일 잉글랜드 덴마크 는 유로 2020 4 강 결승 티켓 두고 격돌 했다', '연장 접전 끝 잉글랜드 21 이겼다', '경기 후 논란 불거졌다', '11 팽팽 하던 연장 전반 12분 라 힘 스털링 드리블 돌파 하던 중 요아킴 멜레 마티아스 옌센 사이 넘어졌다', '심판 은 곧장 페널티 스팟 찍었다', '비디오 판독 실과 의견 나눈 뒤 에도 원심 유지 했다', '페널티킥 얻은 잉글랜드 는 해리 케인 실축 했지만 흐른 볼 밀어 넣어 결승 티켓 따냈다', '장면 두고 갑론 박 펼쳐졌다', '스털링 은 경기 후 인터뷰 명백한 페널티킥 이라고 주장 했지만 전문가 의견 은 달랐다', '조제 모리뉴 AS 로마 감독 아르 센 벵거 전 아스널 감독 은 페널티킥 아니다고 입 모았다', '많은 이야기 흘러나오는 가운데 유로 2020 심판 위원장 로세티 오심 아니라는 입장 내놨다', '로세티 위원장 은 14일 영국 매체 가디언 인터뷰 주심 은 5 번 수비수 주목 했다', '수비수 볼 터치 하지 않았다고 봤다', '멜레 오른쪽 다리 스털링 오른쪽 다리 접촉 한 확인 했다', '접촉 강도 논 할 수 있지만 는 항상 심판 의사결정 과정 중심 되길 바란다고 밝혔다', '로세티 위원장 심판 대표 해 의견 냈지만 오심 이라고 생각 하는 받아들일지는 미지수 다', '스털링 은 평소 에도 다이빙 논란 시 달려왔고 많은 머릿속 다이버 라는 인식 가득하기 때문 이다']
0
유럽 축구 연맹 유

#Extractive summarization - Matchsum
### Dataset & Dataloader 생성

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.utils.data import Dataset, DataLoader,random_split
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer

In [74]:
class CustomDataset(Dataset):
  def __init__(
      self,
      data,
      tokenizer,
      text_max_token_len: int = 512,
      summary_max_token_len : int = 128
  ):
    self.tokenizer = tokenizer
    self.data = data
    self.text_max_token_len = text_max_token_len
    self.summary_max_token_len = summary_max_token_len

  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index):
    cls_token = torch.tensor([0])
    sep_token = torch.tensor([1])
    
    data_row = self.data.iloc[index]
    text = data_row['CONTENT']
    
    total_text_ids = torch.tensor([])
    for sentence in text:
      text_encoding_sentence = self.tokenizer(
          sentence,
          add_special_tokens = True,
          return_tensors = "pt"
      )
      sentence_input_ids = text_encoding_sentence['input_ids'].flatten()
      sentence_input_ids = torch.cat([cls_token,sentence_input_ids,sep_token]) # 각 문장 앞에 cls 토큰 추가, 각 문장 뒤에 sep 토큰 추가
      total_text_ids = torch.cat([total_text_ids,sentence_input_ids])
    
    cur_length = len(total_text_ids)
    if cur_length > self.text_max_token_len:
      total_text_ids = total_text_ids[:self.text_max_token_len] # 길이가 넘치면 자른다
    else:
      padding_list = torch.tensor([3]*(self.text_max_token_len - cur_length)) # 길이가 모자라면 padding token 을 채운다
      total_text_ids = torch.cat([total_text_ids,padding_list])
    
    total_text_ids = total_text_ids.int()

    labels = data_row['TITLE']
    summary_encoding = self.tokenizer(
        labels,
        add_special_tokens = True,
        return_tensors = "pt"
    )

    labels_ids = summary_encoding['input_ids'].flatten()
    
    cur_length = len(labels_ids)
    if cur_length +2 > self.summary_max_token_len:  # 현재 토큰 길이+2(cls,sep) 가 총 길이
      labels_ids = labels_ids[:self.summary_max_token_len-2]
      labels_ids = torch.cat([cls_token,labels_ids,sep_token])
    else : 
      padding_list =torch.tensor([3]*(self.summary_max_token_len-cur_length-2))
      labels_ids =  torch.cat([cls_token,labels_ids,sep_token,padding_list])

    return dict(text_input_ids = total_text_ids, labels_input_ids = labels_ids)


In [58]:
tokenizer = get_kobart_tokenizer()

using cached model. /content/.cache/kobart_base_tokenizer_cased_cf74400bce.zip


In [75]:
whole_dataset = CustomDataset(trimmed_data,tokenizer)

train_set_num = len(trimmed_data)//5*4
train_dataset , test_dataset = random_split(whole_dataset, [train_set_num,len(trimmed_data)-train_set_num])
train_dataloader = DataLoader(train_dataset, batch_size = 8, shuffle=True)
test_dataloader =  DataLoader(test_dataset, batch_size = 8, shuffle=False)

# Model-Load
- Encoder -> KoBART
- GLM 을 제외한 제일 성능 좋은 모델이고 한국어로 train이 되어 있어 선정함

In [ ]:
class MatchSum(nn.Module):
    
    def __init__(self, encoder, candidate_num, hidden_size=768):
        super(MatchSum, self).__init__()
        
        self.hidden_size = hidden_size
        self.candidate_num  = candidate_num
        self.encoder = encoder

    def forward(self, text_id, summary_id):
        
        batch_size = text_id.size(0)
        pad_id = 3 

        # get document embedding
        input_mask = ~(text_id == pad_id)
        out = self.encoder(text_id, attention_mask=input_mask)['hidden_states'][-1] # last layer
        doc_emb = out[:, 0, :]
        assert doc_emb.size() == (batch_size, self.hidden_size) # [batch_size, hidden_size]
        
        # get summary embedding
        input_mask = ~(summary_id == pad_id)
        out = self.encoder(summary_id, attention_mask=input_mask)['hidden_states'][-1] # last layer
        summary_emb = out[:, 0, :]
        assert summary_emb.size() == (batch_size, self.hidden_size) # [batch_size, hidden_size]

        # get summary score
        summary_score = torch.cosine_similarity(summary_emb, doc_emb, dim=-1)

        # get candidate embedding
        candidate_id = get_candidate_id(text_id, self.candidate_num)
        candidate_id = candidate_id.view(-1, candidate_id.size(-1))
        input_mask = ~(candidate_id == pad_id)
        out = self.encoder(candidate_id, attention_mask=input_mask)[0]
        candidate_emb = out[:, 0, :].view(batch_size, self.candidate_num, self.hidden_size)  # [batch_size, candidate_num, hidden_size]
        assert candidate_emb.size() == (batch_size, self.candidate_num, self.hidden_size)
        
        # get candidate score
        doc_emb = doc_emb.unsqueeze(1).expand_as(candidate_emb)
        score = torch.cosine_similarity(candidate_emb, doc_emb, dim=-1) # [batch_size, candidate_num]
        golden_list = torch.argmax(score,dim=1)
        assert score.size() == (batch_size, self.candidate_num)

        return {'score': score, 'summary_score': summary_score, 'golden_summary':candidate_emb[:,golden_list,:]}

In [ ]:
class MarginRankingLoss():      
    
    def __init__(self, margin, score=None, summary_score=None):
        super(MarginRankingLoss, self).__init__()
        self._init_param_map(score=score, summary_score=summary_score)
        self.margin = margin
        self.loss_func = torch.nn.MarginRankingLoss(margin)

    def get_loss(self, score, summary_score):
        
        # equivalent to initializing TotalLoss to 0
        # here is to avoid that some special samples will not go into the following for loop
        ones = torch.ones(score.size()).cuda(score.device)
        loss_func = torch.nn.MarginRankingLoss(0.0)
        TotalLoss = loss_func(score, score, ones)

        # candidate loss
        n = score.size(1)
        for i in range(1, n):
            pos_score = score[:, :-i]
            neg_score = score[:, i:]
            pos_score = pos_score.contiguous().view(-1)
            neg_score = neg_score.contiguous().view(-1)
            ones = torch.ones(pos_score.size()).cuda(score.device)
            loss_func = torch.nn.MarginRankingLoss(self.margin * i)
            TotalLoss += loss_func(pos_score, neg_score, ones)

        # gold summary loss
        pos_score = summary_score.unsqueeze(-1).expand_as(score)
        neg_score = score
        pos_score = pos_score.contiguous().view(-1)
        neg_score = neg_score.contiguous().view(-1)
        ones = torch.ones(pos_score.size()).cuda(score.device)
        loss_func = torch.nn.MarginRankingLoss(0.0)
        TotalLoss += loss_func(pos_score, neg_score, ones)
        
        return TotalLoss

In [ ]:
# https://pypi.org/project/rouge-score/
!pip install rouge-score
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')

In [ ]:
print(scores)

In [ ]:
text_encoding.to(device)
outputs = model(**text_encoding,output_hidden_states = True)

In [ ]:
class rdass:
  def __init__(self,model):
    self.model = model
  
  def __call__(self, text = None, label = None, answer = None):
    vector_text = self.model(text).detach() # vector_d
    vector_label = self.model(label).detach() # vector_r
    vector_answer = self.model(answer).detach() # vector_p

    score_pr = torch.cosine_similarity(vector_answer,vector_label)
    score_pd = torch.cosine_similarity(vector_answer,vector_text)

    return (score_pr+score_pd)/2

In [ ]:
class sbert:        # tokenizer 통합된 version
    def __init__(self,model,device):
        self.model = model
        for param in self.model.parameters():
        param.requires_grad = False
        
        model.to(device)
    
    def __call__(self,inputs_id,attention_mask):
        outputs = self.model(
            inputs_id = inputs_id, 
            token_type_ids = token_type_ids,
            attention_mask = attention_mask
            )
        
        hidden_states = outputs['hidden_states']
        return hidden_states[-1][:,0]

In [ ]:
model = BartModel.from_pretrained(get_pytorch_kobart_model())
summary_model = MatchSum(encoder = model, candidate_num = 5, hidden_size=768) # hidden_size == vocab size?

model_for_eval = sbert(model = ,device)
metric = rdass(model_for_eval)

N_EPOCHS = 3
optimizer = SGD(model.parameters(),lr =0.0001)
scheduler = CosineAnnealingWarmRestarts(optimizer,T_0 = 1,T_mult = 1)

In [ ]:
model.to(device)
summary_model.to(device)

for epoch in range(N_EPOCHS):
    
    print(f"*****Epoch {epoch} Train Start*****")
    print(f"*****Epoch {epoch} Total Step {len(train_dataloader)}*****")
    total_loss, batch_loss, batch_step = 0,0,0

    for step, batch in enumerate(train_dataloader):
        batch_step+=1
        text_input_ids = batch["text_input_ids"].to(device)        
        label_input_ids = batch["label_inputs_ids"].to(device)

        model.zero_grad()
        optimizer.zero_grad()

        # forward
        output = summary_model.forward(text_input_ids, label_input_ids)
        loss = MarginRankingLoss(output["score"],output["summary_score"])

        # loss 계산
        loss.backward()
        # optimizer 업데이트
        optimizer.step()
        # scheduler 업데이트
        scheduler.step()

        batch_loss += loss.item()
        total_loss += loss.item()
        
        rdass_loss = metric(text = text_input_ids,label=  label_input_ids, answer = output['golden_summary'])
        if (step%500 == 0) and (step!=0):
            print(f"Step: {step} Loss: {batch_loss/batch_step:.4f} lr: {optimizer.param_groups[0]['lr']:.4f}")
            # 변수 초기화    
            batch_loss, batch_step = 0,0
    
    print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
    print(f"*****Epoch {epoch} Train Finished*****\n")
    

In [ ]:
def get_candidate_id(tokenizer, cls, sep_id, idx):

    idx_path = join(temp_path, str(idx))
    
    # create some temporary files to calculate ROUGE
    sp.call('mkdir ' + idx_path, shell=True)
    sp.call('mkdir ' + join(idx_path, 'decode'), shell=True)
    sp.call('mkdir ' + join(idx_path, 'reference'), shell=True)
    
    # load data
    data = {}
    data['text'] = original_data[idx]['text']
    data['summary'] = original_data[idx]['summary']
    
    # write reference summary to temporary files
    ref_dir = join(idx_path, 'reference')
    with open(join(ref_dir, '0.ref'), 'w') as f:
        for sentence in data['summary']:
            print(sentence, file=f)

    # get candidate summaries
    # here is for CNN/DM: truncate each document into the 5 most important sentences (using BertExt), 
    # then select any 2 or 3 sentences to form a candidate summary, so there are C(5,2)+C(5,3)=20 candidate summaries.
    # if you want to process other datasets, you may need to adjust these numbers according to specific situation.
    sent_id = sent_ids[idx]['sent_id'][:5]
    indices = list(combinations(sent_id, 2))
    indices += list(combinations(sent_id, 3))
    if len(sent_id) < 2:
        indices = [sent_id]
    
    # get ROUGE score for each candidate summary and sort them in descending order
    score = []
    for i in indices:
        i = list(i)
        i.sort()
        # write dec
        dec = []
        for j in i:
            sent = data['text'][j]
            dec.append(sent)
        score.append((i, get_rouge(idx_path, dec)))
    score.sort(key=lambda x : x[1], reverse=True)
    
    # write candidate indices and score
    data['ext_idx'] = sent_id
    data['indices'] = []
    data['score'] = []
    for i, R in score:
        data['indices'].append(list(map(int, i)))
        data['score'].append(R)

    # tokenize and get candidate_id
    candidate_summary = []
    for i in data['indices']:
        cur_summary = [cls]
        for j in i:
            cur_summary += data['text'][j].split()
        cur_summary = cur_summary[:MAX_LEN]
        cur_summary = ' '.join(cur_summary)
        candidate_summary.append(cur_summary)
    
    data['candidate_id'] = []
    for summary in candidate_summary:
        token_ids = tokenizer.encode(summary, add_special_tokens=False)[:(MAX_LEN - 1)]
        token_ids += sep_id
        data['candidate_id'].append(token_ids)
    
    # tokenize and get text_id
    text = [cls]
    for sent in data['text']:
        text += sent.split()
    text = text[:MAX_LEN]
    text = ' '.join(text)
    token_ids = tokenizer.encode(text, add_special_tokens=False)[:(MAX_LEN - 1)]
    token_ids += sep_id
    data['text_id'] = token_ids
    
    # tokenize and get summary_id
    summary = [cls]
    for sent in data['summary']:
        summary += sent.split()
    summary = summary[:MAX_LEN]
    summary = ' '.join(summary)
    token_ids = tokenizer.encode(summary, add_special_tokens=False)[:(MAX_LEN - 1)]
    token_ids += sep_id
    data['summary_id'] = token_ids
    
    # write processed data to temporary file
    processed_path = join(temp_path, 'processed')
    with open(join(processed_path, '{}.json'.format(idx)), 'w') as f:
        json.dump(data, f, indent=4) 
    
    sp.call('rm -r ' + idx_path, shell=True)